
 # FlowGenie: Natural-Language Workflow Automation Designer 🧠⚙️

 Track: Enterprise Agents  
 Tech: Google ADK (google-adk), Gemini, Multi-Agent, Tools, Evaluation, Sessions, Memory

 Features:
 - Natural language → automation workflows (JSON)
 - Planner, Evaluator, Executor (tool routing)
 - Tool agents for Slack, Sheets, Gmail, Calendar
 - Persistent sessions via SQLite (DatabaseSessionService)
 - Chat agent with long-term memory (InMemoryMemoryService)
 - Router that decides: "chat" vs "automation"

 Behavior:
 - Normal talk → chat agent (with memory)
 - Automation requests → FlowGenie pipeline



## 1. Setup & Dependencies

Install core libraries and configure environment variables:

- `google-adk`
- `google-genai`
- `python-dotenv`
- Google APIs (Sheets, Gmail, Calendar)


In [ ]:

!pip install -q python-dotenv
!pip install -q -U google-adk
!pip install -q google-api-python-client google-auth google-auth-httplib2 requests
!pip install -q aiosqlite


✅ Using model: gemini-2.5-flash-lite


In [ ]:

from dotenv import load_dotenv
import os
import json
from typing import Dict, Any, List

load_dotenv(override=True)

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in environment. Please set it before running.")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

MODEL_NAME = "gemini-2.5-flash-lite"
print("✅ Using model:", MODEL_NAME)



In [ ]:

import asyncio
from datetime import datetime

from google.genai import types

from google.adk.agents import Agent, LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import preload_memory

print("✅ ADK imports ready.")



✅ ADK imports ready.


In [ ]:

import base64
from email.mime.text import MIMEText
import requests
from google.oauth2 import service_account
from googleapiclient.discovery import build

USE_REAL_APIS = os.getenv("USE_REAL_APIS", "true").lower() == "true"
print("✅ Real API usage:", USE_REAL_APIS)

def get_service_account_credentials(scopes: List[str]):
    """
    Build service account credentials from GOOGLE_SERVICE_ACCOUNT_JSON.
    Accepts:
      - JSON string
      - path to JSON file
      - Python dict string (fallback via ast.literal_eval)
    """
    json_str = os.getenv("GOOGLE_SERVICE_ACCOUNT_JSON")
    if not json_str:
        raise RuntimeError("GOOGLE_SERVICE_ACCOUNT_JSON env var not set.")

    # If env value is a path, load file
    if os.path.exists(json_str):
        with open(json_str, "r", encoding="utf-8") as f:
            json_str = f.read()

    try:
        info = json.loads(json_str)
    except Exception:
        import ast
        try:
            info = ast.literal_eval(json_str)
        except Exception as e:
            raise RuntimeError(
                "Failed to parse GOOGLE_SERVICE_ACCOUNT_JSON. "
                "Provide a valid JSON string or a path to a JSON file."
            ) from e

    creds = service_account.Credentials.from_service_account_info(info, scopes=scopes)

    delegated_user = os.getenv("GMAIL_SENDER_EMAIL")
    if delegated_user:
        creds = creds.with_subject(delegated_user)

    return creds


✅ Real API usage: True


## 2. Workflow Schema and Core Automation Agents

Define the **JSON schema** that represents a workflow and create:

- `planner_agent`: turns natural language into workflow JSON
- `evaluator_agent`: scores the workflow for quality, risks, and improvements


In [ ]:

WORKFLOW_JSON_GUIDE = """
You are designing an automation workflow.

Always output a JSON object with this structure:

{
  "name": "<short human-friendly name>",
  "trigger": {
    "type": "<trigger_type>",
    "source": "<where the event originates>",
    "schedule": "<cron or time-based trigger, or null>",
    "conditions": [
      "<optional condition 1>",
      "<optional condition 2>"
    ]
  },
  "actions": [
    {
      "type": "<action_type>",
      "target": "<system or destination>",
      "description": "<what this step does>",
      "inputs": [
        "<critical input or data needed>"
      ]
    }
  ]
}

Rules:
- Use lowercase snake_case for types when possible.
- If something is unknown, put null or an empty list instead of guessing wildly.
- The workflow should be as simple as possible while still solving the user's request.
"""

planner_agent = Agent(
    name="workflow_planner",
    model=MODEL_NAME,
    description="Plans automation workflows from natural-language requests.",
    instruction=(
        "You are an expert workflow automation designer.\n"
        "Given a user's natural-language request, generate exactly ONE automation workflow.\n"
        "DO NOT return a list of workflows.\n"
        "DO NOT return multiple possible interpretations.\n"
        "Return ONLY ONE JSON object that follows this schema:\n"
        f"{WORKFLOW_JSON_GUIDE}\n\n"
        "Respond with ONLY the JSON object. Never return a list or surrounding quotes."
    ),
)

evaluator_agent = Agent(
    name="workflow_evaluator",
    model=MODEL_NAME,
    description="Evaluates workflow quality, safety, and completeness.",
    instruction=(
        "You are evaluating an automation workflow JSON.\n"
        "Check for:\n"
        "- Clarity of trigger\n"
        "- Completeness of actions\n"
        "- Edge cases and failure modes\n"
        "- Privacy or safety concerns\n\n"
        "Return a JSON object with:\n"
        "{\n"
        '  \"overall_score\": <0-10>,\n'
        '  \"verdict\": \"<ACCEPT or IMPROVE>\",\n'
        '  \"strengths\": [\"...\"],\n'
        '  \"risks\": [\"...\"],\n'
        '  \"suggested_changes\": [\"...\"]\n'
        "}\n"
        "Be concise but specific. Respond with ONLY JSON."
    ),
)

print("✅ Planner & Evaluator agents created.")



✅ Planner & Evaluator agents created.


In [ ]:

def simulate_workflow_execution(workflow: Dict[str, Any]) -> Dict[str, Any]:
    """
    Pure Python simulation: does NOT call real APIs.
    """
    name = workflow.get("name", "unnamed_workflow")
    actions = workflow.get("actions", [])

    log = []
    for idx, action in enumerate(actions, start=1):
        log.append(
            {
                "step": idx,
                "type": action.get("type"),
                "target": action.get("target"),
                "status": "success",
                "timestamp": datetime.utcnow().isoformat() + "Z",
                "note": f"Simulated execution of step {idx}",
            }
        )

    return {
        "workflow_name": name,
        "total_steps": len(actions),
        "completed_steps": len(actions),
        "status": "completed" if actions else "no_actions",
        "log": log,
    }



## 3. Real Tools: Slack, Sheets, Gmail, Calendar

These pure Python functions wrap real APIs:

- `slack_send_notification`
- `sheets_append_row`
- `gmail_send_email`
- `calendar_create_event`

Each tool:
- Reads configuration from environment variables
- Can run in **real mode** (`USE_REAL_APIS=true`) or **safe no-op mode** for demos


In [ ]:

from urllib.parse import urlparse

def slack_send_notification(channel: str, message: str) -> Dict[str, Any]:
    """
    Send a Slack notification via webhook. Uses SLACK_WEBHOOK_URL.
    """
    webhook_url = os.getenv("SLACK_WEBHOOK_URL")
    if not USE_REAL_APIS or not webhook_url:
        print(f"[SLACK NO-OP] Channel={channel} | Message={message}")
        return {
            "channel": channel,
            "message": message,
            "status": "skipped",
            "reason": "USE_REAL_APIS is false or SLACK_WEBHOOK_URL not set",
        }

    # Validate URL
    try:
        parsed = urlparse(webhook_url)
        if not parsed.scheme or not parsed.netloc:
            raise ValueError("invalid")
    except Exception:
        return {
            "channel": channel,
            "message": message,
            "status": "error",
            "reason": "SLACK_WEBHOOK_URL is malformed or missing scheme.",
            "webhook_preview": (webhook_url[:50] + "...") if webhook_url else "<empty>",
        }

    payload = {"text": message}
    try:
        resp = requests.post(webhook_url, json=payload, timeout=10)
    except requests.RequestException as e:
        return {
            "channel": channel,
            "message": message,
            "status": "error",
            "reason": f"request_exception: {type(e).__name__}: {e}",
        }

    return {
        "channel": channel,
        "message": message,
        "http_status": resp.status_code,
        "status": "sent" if resp.status_code in (200, 204) else "error",
        "response_text": resp.text[:200],
    }


def sheets_append_row(row_values: List[str]) -> Dict[str, Any]:
    """
    Append a row to Google Sheet using service account.
    Uses SHEETS_SPREADSHEET_ID and SHEETS_TAB_NAME env vars.
    """
    spreadsheet_id = os.getenv("SHEETS_SPREADSHEET_ID")
    sheet_name = os.getenv("SHEETS_TAB_NAME", "Sheet1")

    if not USE_REAL_APIS:
        print(f"[SHEETS NO-OP] Spreadsheet={spreadsheet_id} | Sheet={sheet_name} | Row={row_values}")
        return {
            "spreadsheet_id": spreadsheet_id,
            "sheet_name": sheet_name,
            "row_values": row_values,
            "status": "skipped",
            "reason": "USE_REAL_APIS is false",
        }

    scopes = ["https://www.googleapis.com/auth/spreadsheets"]
    creds = get_service_account_credentials(scopes)
    service = build("sheets", "v4", credentials=creds)

    range_ = f"{sheet_name}!A1"
    body = {"values": [row_values]}

    result = (
        service.spreadsheets()
        .values()
        .append(
            spreadsheetId=spreadsheet_id,
            range=range_,
            valueInputOption="USER_ENTERED",
            insertDataOption="INSERT_ROWS",
            body=body,
        )
        .execute()
    )

    return {
        "spreadsheet_id": spreadsheet_id,
        "sheet_name": sheet_name,
        "row_values": row_values,
        "status": "appended",
        "updates": result.get("updates", {}),
    }


def gmail_send_email(to: str, subject: str, body: str) -> Dict[str, Any]:
    """
    Send an email using Gmail API and a service account.
    Uses GMAIL_SENDER_EMAIL env var as sender.
    """
    sender = os.getenv("GMAIL_SENDER_EMAIL") or "example@example.com"

    if not USE_REAL_APIS:
        print(f"[GMAIL NO-OP] From={sender} | To={to} | Subject={subject}")
        return {
            "from": sender,
            "to": to,
            "subject": subject,
            "status": "skipped",
            "reason": "USE_REAL_APIS is false",
        }

    scopes = ["https://www.googleapis.com/auth/gmail.send"]
    creds = get_service_account_credentials(scopes)
    service = build("gmail", "v1", credentials=creds)

    msg = MIMEText(body)
    msg["to"] = to
    msg["from"] = sender
    msg["subject"] = subject

    raw = base64.urlsafe_b64encode(msg.as_bytes()).decode("utf-8")
    message = {"raw": raw}

    sent = (
        service.users()
        .messages()
        .send(userId="me", body=message)
        .execute()
    )

    return {
        "from": sender,
        "to": to,
        "subject": subject,
        "status": "sent",
        "message_id": sent.get("id"),
    }


def calendar_create_event(calendar_id: str, title: str, start_time: str, end_time: str) -> Dict[str, Any]:
    """
    Create a calendar event using Calendar API.
    """
    if not USE_REAL_APIS:
        print(f"[CALENDAR NO-OP] Calendar={calendar_id} | Title={title} | {start_time} -> {end_time}")
        return {
            "calendar_id": calendar_id,
            "title": title,
            "start_time": start_time,
            "end_time": end_time,
            "status": "skipped",
            "reason": "USE_REAL_APIS is false",
        }

    scopes = ["https://www.googleapis.com/auth/calendar"]
    creds = get_service_account_credentials(scopes)
    service = build("calendar", "v3", credentials=creds)

    event = {
        "summary": title,
        "start": {"dateTime": start_time},
        "end": {"dateTime": end_time},
    }

    created = (
        service.events()
        .insert(calendarId=calendar_id, body=event)
        .execute()
    )

    return {
        "calendar_id": calendar_id,
        "title": title,
        "start_time": start_time,
        "end_time": end_time,
        "status": "created",
        "event_id": created.get("id"),
    }

print("✅ Real tools defined.")


✅ Real tools defined.


## 4. Tool Agents (A2A Pattern)

Wrap raw tools inside **LLM-powered agents**:

- `slack_agent`
- `sheets_agent`
- `gmail_agent`
- `calendar_agent`

These agents:
- Parse high-level parameters
- Call the underlying Python tools
- Return **only the tool JSON result** (no extra prose)


In [ ]:

slack_agent = Agent(
    name="slack_agent",
    model=MODEL_NAME,
    description="Agent that sends Slack notifications via a tool.",
    instruction=(
        "You send notifications to Slack channels.\n"
        "ALWAYS call the tool slack_send_notification(channel=<channel>, message=<message>).\n"
        "Return ONLY the JSON returned by the tool, no extra text."
    ),
    tools=[slack_send_notification],
)

sheets_agent = Agent(
    name="sheets_agent",
    model=MODEL_NAME,
    description="Agent that appends rows to a Google Sheet via a tool.",
    instruction=(
        "You receive a request to append data to a spreadsheet.\n"
        "Extract ONLY the row_values from the input.\n"
        "ALWAYS call sheets_append_row(row_values=<list of strings>).\n"
        "sheet_name and spreadsheet_id will be loaded from environment inside the tool.\n"
        "Return ONLY the JSON returned by the tool."
    ),
    tools=[sheets_append_row],
)

gmail_agent = Agent(
    name="gmail_agent",
    model=MODEL_NAME,
    description="Agent that sends emails via a tool.",
    instruction=(
        "You send emails.\n"
        "ALWAYS call gmail_send_email(to=<email>, subject=<subject>, body=<body>).\n"
        "Return ONLY the JSON returned by the tool."
    ),
    tools=[gmail_send_email],
)

calendar_agent = Agent(
    name="calendar_agent",
    model=MODEL_NAME,
    description="Agent that creates calendar events via a tool.",
    instruction=(
        "You create calendar events.\n"
        "ALWAYS call calendar_create_event(calendar_id=<calendar_id>, title=<title>, "
        "start_time=<start>, end_time=<end>).\n"
        "Return ONLY the JSON returned by the tool."
    ),
    tools=[calendar_create_event],
)

print("✅ Tool agents created.")


✅ Tool agents created.


## 5. Sessions & Memory

Use:

- `DatabaseSessionService` with SQLite (`flowgenie_sessions.db`) for **persistent sessions**
- `InMemoryMemoryService` for **long-term memory** (course requirement)

Also define:
- `auto_save_to_memory` callback → automatically saves each completed agent turn into memory


In [ ]:

# SQLite async URL (requires aiosqlite)
db_url = "sqlite+aiosqlite:///flowgenie_sessions.db"
session_service = DatabaseSessionService(db_url=db_url)

# Long-term memory (lives in process only)
memory_service = InMemoryMemoryService()

APP_NAME = "flowgenie"
USER_ID = "demo_user"

print("✅ SQLite session service & InMemory memory service initialized.")


✅ SQLite session service & InMemory memory service initialized.


In [ ]:

async def auto_save_to_memory(callback_context):
    """
    Automatically save entire session to memory after each agent turn.
    Only effective for runners that were given memory_service.
    """
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )

print("✅ auto_save_to_memory callback defined.")


✅ auto_save_to_memory callback defined.


## 6. Runners for Planner, Evaluator, and Tool Agents

Create `Runner` instances for:

- Planner (`planner_runner`)
- Evaluator (`evaluator_runner`)
- Slack / Sheets / Gmail / Calendar tool agents

All share:
- The same `APP_NAME`
- The same `session_service` (SQLite)
- The same `memory_service`


In [ ]:

planner_runner = Runner(
    agent=planner_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

evaluator_runner = Runner(
    agent=evaluator_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

slack_runner = Runner(
    agent=slack_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

sheets_runner = Runner(
    agent=sheets_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

gmail_runner = Runner(
    agent=gmail_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

calendar_runner = Runner(
    agent=calendar_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Automation runners initialized.")


✅ Automation runners initialized.


## 7. Executor Agent: From Workflow JSON → Tool Action Plan

`executor_agent`:

- Reads a **workflow JSON** (with actions)
- Chooses which agent should run each action (`slack_agent`, `sheets_agent`, etc.)
- Builds a structured JSON:

```json
{
  "plan": [...],
  "simulation": {...},
  "summary": "..."
}


In [ ]:

executor_agent = Agent(
    name="workflow_executor",
    model=MODEL_NAME,
    description="Analyzes the workflow JSON and creates a structured step-by-step execution plan for tool agents.",
    instruction=(
        "You receive a workflow JSON object with a list of actions.\n"
        "Your job is to produce a structured PLAN listing which tool agent should execute each action.\n\n"
        "For each action in the workflow, extract the input data and create parameters for the tool agent.\n"
        "For sheets: Extract ONLY row values from action.inputs or description and pass as:\n"
        '  \"parameters\": {\"row_values\": [<list of strings>]}\n\n'
        "For each action, output an item in the plan like this:\n"
        "{\n"
        '  \"action_index\": <1-based index>,\n'
        '  \"agent\": \"slack_agent | sheets_agent | gmail_agent | calendar_agent | skipped\",\n'
        '  \"parameters\": {\n'
        '      ... only the essential parameters ...\n'
        '  }\n'
        "}\n\n"
        "Rules for matching:\n"
        "- Slack if type contains 'slack', 'notification', 'alert'\n"
        "- Sheets if type contains 'sheet', 'spreadsheet', 'row'\n"
        "- Gmail if type contains 'email', 'gmail', 'mail'\n"
        "- Calendar if type contains 'calendar', 'event', 'schedule'\n"
        "- Otherwise mark as skipped\n\n"
        "Return ONLY this JSON structure:\n"
        "{\n"
        '  \"plan\": [... list of tool call definitions ...],\n'
        '  \"simulation\": {\n'
        '    \"workflow_name\": \"<name>\",\n'
        '    \"total_steps\": <count>,\n'
        '    \"completed_steps\": <count>,\n'
        '    \"status\": \"completed\",\n'
        '    \"log\": []\n'
        '  },\n'
        '  \"summary\": \"<human friendly short summary>\"\n'
        "}\n"
        "Do not wrap response in quotes or code fences. Output ONLY JSON."
    ),
)

executor_runner = Runner(
    agent=executor_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Executor agent + runner initialized.")


✅ Executor agent + runner initialized.



(Then your `parse_json_output` helper and `execute_action_plan` functions.)

### 8. Core FlowGenie Pipeline

```md
## 8. FlowGenie Pipeline (Automation Mode)

`run_flowgenie(user_prompt)` performs:

1. **Plan** → `planner_runner`
2. **Evaluate** → `evaluator_runner`
3. **Execute plan** → `executor_runner` + tool agents
4. Returns a full JSON object:

```json
{
  "workflow": {...},
  "evaluation": {...},
  "action_plan": [...],
  "action_results": [...],
  "simulation": {...},
  "summary": "..."
}


In [ ]:

def parse_json_output(raw):
    """
    Robust JSON extraction for ADK responses.
    Handles:
    - Lists of Event objects
    - function_response parts (tools)
    - plain text JSON
    - triple-quoted JSON
    Fixes common issues like \"I\\'m ...\" invalid escapes.
    """
    import json
    import re
    from json.decoder import JSONDecodeError

    def _try_json_loads(text: str):
        # Fix invalid \' escape (not allowed in JSON)
        fixed = text.replace("\\'", "'")
        return json.loads(fixed)

    def extract_from_event(ev):
        try:
            parts = ev.content.parts
        except Exception:
            return None

        # Prefer tool function_response
        for part in parts:
            fr = getattr(part, "function_response", None)
            if fr is not None:
                return getattr(fr, "response", None)

        # Otherwise collect text parts
        texts = []
        for part in parts:
            t = getattr(part, "text", None)
            if t:
                texts.append(t)
        if texts:
            return "\n".join(texts)
        return None

    # 1) If list → iterate events
    if isinstance(raw, list):
        collected_texts = []
        for ev in raw:
            if isinstance(ev, dict):
                return ev
            extracted = extract_from_event(ev)
            if isinstance(extracted, (dict, list)):
                return extracted
            if isinstance(extracted, str) and extracted.strip():
                collected_texts.append(extracted)

        if collected_texts:
            raw = "\n".join(collected_texts)
        else:
            raise ValueError(f"No valid JSON found in list: {raw}")

    # 2) Single Event
    if hasattr(raw, "content") and hasattr(raw.content, "parts"):
        extracted = extract_from_event(raw)
        if isinstance(extracted, (dict, list)):
            return extracted
        if isinstance(extracted, str) and extracted.strip():
            raw = extracted
        else:
            raw = ""

    # 3) Already dict/list → done
    if isinstance(raw, (dict, list)):
        return raw

    # 4) Fallback for objects with .text / .data
    if hasattr(raw, "text"):
        raw = raw.text
    elif hasattr(raw, "data"):
        raw = raw.data

    # 5) Expect string now
    if not isinstance(raw, str):
        raise ValueError(f"Unsupported type in parse_json_output: {type(raw).__name__}")

    txt = raw.strip()

    # Strip markdown fences ```json ... ```
    if txt.startswith("```"):
        txt = txt.strip("`")
        if txt.startswith("json"):
            parts = txt.split("\n", 1)
            txt = parts[1] if len(parts) > 1 else ""

    # Strip triple-quoted wrappers """{...}""" or '''{...}'''
    if (txt.startswith('"""') and txt.endswith('"""')) or (txt.startswith("'''") and txt.endswith("'''")):
        txt = txt.strip().lstrip('\'"').rstrip('\'"').strip()

    # Try direct JSON
    try:
        return _try_json_loads(txt)
    except Exception:
        pass

    # Try extract first {...} region
    match = re.search(r"\{[\s\S]*\}", txt)
    if match:
        candidate = match.group(0)
        try:
            return _try_json_loads(candidate)
        except Exception:
            pass

    # Last resort: return raw text
    return txt

print("✅ parse_json_output defined.")


✅ parse_json_output defined.


In [ ]:

async def execute_action_plan(plan):
    results = []

    for step in plan:
        agent_name = step.get("agent")
        params = step.get("parameters", {})
        idx = step.get("action_index", None)

        if agent_name == "slack_agent":
            resp = await slack_runner.run_debug(json.dumps(params), verbose=False)
        elif agent_name == "sheets_agent":
            resp = await sheets_runner.run_debug(json.dumps(params), verbose=False)
        elif agent_name == "gmail_agent":
            resp = await gmail_runner.run_debug(json.dumps(params), verbose=False)
        elif agent_name == "calendar_agent":
            resp = await calendar_runner.run_debug(json.dumps(params), verbose=False)
        else:
            results.append({
                "action_index": idx,
                "agent": agent_name,
                "result": {"status": "skipped_no_matching_agent"},
            })
            continue

        # Parse tool result safely
        try:
            tool_result = parse_json_output(resp)
        except Exception as e:
            tool_result = {"error": str(e), "raw_response": str(resp)}

        results.append({
            "action_index": idx,
            "agent": agent_name,
            "result": tool_result,
        })

    return results

print("✅ execute_action_plan defined.")


✅ execute_action_plan defined.


In [ ]:

async def run_flowgenie(user_prompt: str):
    """
    Automation-only pipeline:
    1) Plan workflow
    2) Evaluate workflow
    3) Ask executor to build tool-call plan
    4) Execute plan via tool agents
    """
    # 1) Planner
    plan_resp = await planner_runner.run_debug(user_prompt, verbose=True)
    workflow = parse_json_output(plan_resp)

    # 2) Evaluator
    eval_resp = await evaluator_runner.run_debug(json.dumps(workflow), verbose=True)
    evaluation = parse_json_output(eval_resp)

    # 3) Executor → plan JSON
    exec_resp = await executor_runner.run_debug(json.dumps(workflow), verbose=True)
    exec_obj = parse_json_output(exec_resp)

    if isinstance(exec_obj, str):
        # If still plain text, we couldn't parse JSON
        raise ValueError(f"Executor did not return JSON:\n{exec_obj}")

    if not isinstance(exec_obj, dict):
        raise ValueError(f"Executor returned non-dict object:\n{exec_obj}")

    action_plan = exec_obj.get("plan", [])
    simulation = exec_obj.get("simulation")

    # 4) Execute actions
    tool_results = await execute_action_plan(action_plan)

    return {
        "workflow": workflow,
        "evaluation": evaluation,
        "action_plan": action_plan,
        "action_results": tool_results,
        "simulation": simulation,
        "summary": exec_obj.get("summary"),
    }

print("✅ run_flowgenie defined.")



✅ run_flowgenie defined.




## 9. Intent Router: Chat vs Automation

`router_agent`:

- Reads the user prompt
- Responds with **only** one word: `"chat"` or `"automation"`

This keeps normal conversation separate from workflow automation.


In [ ]:

router_agent = Agent(
    name="intent_router",
    model=MODEL_NAME,
    instruction=(
        "You are an intent classifier.\n"
        "Only classify as 'automation' if the user is explicitly asking to perform an automated ACTION "
        "such as: send an email, send a Slack notification, append a spreadsheet row, schedule a calendar event, "
        "create an automation, integrate systems, or build a workflow.\n\n"
        "If the user is simply talking about themselves, their job, hobbies, greetings, opinions, questions, "
        "or anything that is not a clear automation request → classify as 'chat'.\n\n"
        "Return only ONE word with no punctuation:\n"
        "automation\n"
        "chat"
    )
)


router_runner = Runner(
    agent=router_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Router agent initialized.")



✅ Router agent initialized.


## 10. Chat Agent with Long-Term Memory

`chat_agent`:

- Uses `preload_memory` to load relevant memories before each turn
- Uses `auto_save_to_memory` callback to store new information
- Acts as a personal assistant (non-automation talk)

`chat_runner` connects the agent with the same SQLite sessions and memory service.


In [ ]:

chat_agent = LlmAgent(
    model=MODEL_NAME,
    name="chat_assistant",
    instruction=(
        "You are a helpful personal assistant.\n"
        "Have natural conversations, remember user's preferences, and answer clearly.\n"
        "You are NOT responsible for building automation workflows; if user explicitly asks "
        "for automations, they will be routed to a different agent."
    ),
    tools=[preload_memory],                  # proactively load relevant memory
    after_agent_callback=auto_save_to_memory # automatically save each turn
)

chat_runner = Runner(
    agent=chat_agent,
    app_name=f"{APP_NAME}_chat",
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Chat agent with memory initialized.")



✅ Chat agent with memory initialized.


## 11. Unified Entry Point: handle_user_input

`handle_user_input(user_prompt, session_id)`:

1. Uses the router to decide:
   - `"chat"` → send to `chat_runner`
   - `"automation"` → send to `run_flowgenie`
2. Returns:

- Chat:
  ```json
  {"mode": "chat", "reply": "..."}


In [ ]:

async def handle_user_input(user_prompt: str, session_id: str = "default_session"):
    """
    Single entrypoint for your app.
    - Uses router to classify: chat vs automation
    - Automation → FlowGenie pipeline
    - Chat → chat agent with memory

    Returns:
      - Chat:       {"mode": "chat", "reply": "<text>"}
      - Automation: {"mode": "automation", ...FlowGenie result...}
    """
    # 1) Route intent
    route_events = await router_runner.run_debug(user_prompt, verbose=False)
    # run_debug returns a list of events; grab last with text
    route_text = ""
    if isinstance(route_events, list):
        for ev in route_events:
            if getattr(ev, "content", None) and ev.content.parts:
                t = ev.content.parts[0].text or ""
                if t.strip():
                    route_text = t.strip().lower()
    else:
        if getattr(route_events, "content", None) and route_events.content.parts:
            route_text = (route_events.content.parts[0].text or "").strip().lower()
        else:
            route_text = str(route_events).strip().lower()

    # 2) Automation path
    if "automation" in route_text:
        flowgenie_result = await run_flowgenie(user_prompt)
        flowgenie_result["mode"] = "automation"
        return flowgenie_result

    # 3) Chat path
    # Ensure chat session exists in DB so Runner won't complain
    try:
        await session_service.create_session(
            app_name=chat_runner.app_name,
            user_id=USER_ID,
            session_id=session_id,
            state={},
        )
    except Exception:
        # Already exists → ignore
        pass

    last_text = ""
    async for event in chat_runner.run_async(
        user_id=USER_ID,
        session_id=session_id,
        new_message=types.Content(role="user", parts=[types.Part(text=user_prompt)]),
    ):
        if event.content and event.content.parts and event.content.parts[0].text:
            last_text = event.content.parts[0].text

    return {
        "mode": "chat",
        "reply": last_text,
    }

print("✅ handle_user_input() ready.")


✅ handle_user_input() ready.


## 13. Quick Tests

### 13.1 Automation Example – Slack

Test an automation-style request:

- Router classifies as `"automation"`
- FlowGenie plans + evaluates + executes a Slack notification

### 13.2 Chat Example – Normal Conversation

Test a normal chat:

- Router classifies as `"chat"`
- Chat agent responds and stores user facts in memory
- Later questions can retrieve that memory


In [ ]:

# # Example 1: Automation request
prompt = "Add a row to my spreadsheet with: (323, sinu, login issue, failed to update details)"
result = await handle_user_input(prompt, session_id="user_1")
print(json.dumps(result, indent=2))



 ### Continue session: debug_session_id

User > Add a row to my spreadsheet with: (323, sinu, hehe, samajh gya)


Event from an unknown agent: intent_router, event id: e9584850-688a-44a5-954e-8f780db1a2cb
Event from an unknown agent: intent_router, event id: cfb720be-c37a-4158-8334-e767b3ff2171
Event from an unknown agent: intent_router, event id: 8d73f299-c472-4892-a942-c3c2b613d1fd
Event from an unknown agent: slack_agent, event id: 693dbacd-c9a4-448e-80d1-7c34f5131720
Event from an unknown agent: slack_agent, event id: 1c6cdb65-d579-415e-b7b7-6d65ff2b0fb8
Event from an unknown agent: slack_agent, event id: b2adc65b-e6b5-4acb-b3b1-b93909b19c90
Event from an unknown agent: workflow_executor, event id: 5cde7bb9-7731-47b4-b318-1333a1929b7f
Event from an unknown agent: workflow_evaluator, event id: 251986e4-65ee-4dd5-9f07-d70bb4269da7


intent_router > automation

 ### Continue session: debug_session_id

User > Add a row to my spreadsheet with: (323, sinu, hehe, samajh gya)


Event from an unknown agent: workflow_planner, event id: 30b271ea-3173-4ea7-ad5b-27606c41c547
Event from an unknown agent: intent_router, event id: e9584850-688a-44a5-954e-8f780db1a2cb
Event from an unknown agent: intent_router, event id: cfb720be-c37a-4158-8334-e767b3ff2171
Event from an unknown agent: intent_router, event id: 8d73f299-c472-4892-a942-c3c2b613d1fd
Event from an unknown agent: slack_agent, event id: 693dbacd-c9a4-448e-80d1-7c34f5131720
Event from an unknown agent: slack_agent, event id: 1c6cdb65-d579-415e-b7b7-6d65ff2b0fb8
Event from an unknown agent: slack_agent, event id: b2adc65b-e6b5-4acb-b3b1-b93909b19c90
Event from an unknown agent: workflow_executor, event id: 5cde7bb9-7731-47b4-b318-1333a1929b7f


workflow_planner > ```json
{
  "name": "log_misc_issue",
  "trigger": {
    "type": "intent_router",
    "source": "intent_router",
    "schedule": null,
    "conditions": [
      "intent_router said: automation"
    ]
  },
  "actions": [
    {
      "type": "add_row",
      "target": "spreadsheet",
      "description": "Add a row to the spreadsheet with issue details.",
      "inputs": [
        "323",
        "sinu",
        "hehe",
        "samajh gya"
      ]
    }
  ]
}
```

 ### Continue session: debug_session_id

User > {"name": "log_misc_issue", "trigger": {"type": "intent_router", "source": "intent_router", "schedule": null, "conditions": ["intent_router said: automation"]}, "actions": [{"type": "add_row", "target": "spreadsheet", "description": "Add a row to the spreadsheet with issue details.", "inputs": ["323", "sinu", "hehe", "samajh gya"]}]}


Event from an unknown agent: workflow_evaluator, event id: a549c3f5-905a-4f6a-8361-596370a290fa
Event from an unknown agent: workflow_planner, event id: 30b271ea-3173-4ea7-ad5b-27606c41c547
Event from an unknown agent: intent_router, event id: e9584850-688a-44a5-954e-8f780db1a2cb
Event from an unknown agent: intent_router, event id: cfb720be-c37a-4158-8334-e767b3ff2171
Event from an unknown agent: intent_router, event id: 8d73f299-c472-4892-a942-c3c2b613d1fd
Event from an unknown agent: slack_agent, event id: 693dbacd-c9a4-448e-80d1-7c34f5131720
Event from an unknown agent: slack_agent, event id: 1c6cdb65-d579-415e-b7b7-6d65ff2b0fb8
Event from an unknown agent: slack_agent, event id: b2adc65b-e6b5-4acb-b3b1-b93909b19c90


workflow_evaluator > ```json
{
  "overall_score": 7,
  "verdict": "IMPROVE",
  "strengths": [
    "Clear Trigger: The workflow is triggered by the specific phrase 'intent_router said: automation', making its initiation explicit and understandable.",
    "Specific Action: The `add_row` action is well-defined, including all necessary inputs for the spreadsheet entry."
  ],
  "risks": [
    "Lack of Error Handling: The workflow does not include explicit error handling for the `add_row` operation. Failures (e.g., spreadsheet unavailability, invalid data) might occur without notification or retry.",
    "Hardcoded Data: The inputs ('323', 'sinu', 'hehe', 'samajh gya') are hardcoded, limiting the workflow's flexibility and reusability for different scenarios or data types.",
    "Broad Trigger Condition: The trigger 'intent_router said: automation' is very general. It could potentially activate this workflow for unrelated 'automation' intents, leading to unintended data logging.",
    "Vague

Event from an unknown agent: workflow_executor, event id: a1f729d2-53a4-4de2-b082-8b271a220983
Event from an unknown agent: workflow_evaluator, event id: a549c3f5-905a-4f6a-8361-596370a290fa
Event from an unknown agent: workflow_planner, event id: 30b271ea-3173-4ea7-ad5b-27606c41c547
Event from an unknown agent: intent_router, event id: e9584850-688a-44a5-954e-8f780db1a2cb
Event from an unknown agent: intent_router, event id: cfb720be-c37a-4158-8334-e767b3ff2171
Event from an unknown agent: intent_router, event id: 8d73f299-c472-4892-a942-c3c2b613d1fd
Event from an unknown agent: slack_agent, event id: 693dbacd-c9a4-448e-80d1-7c34f5131720
Event from an unknown agent: slack_agent, event id: 1c6cdb65-d579-415e-b7b7-6d65ff2b0fb8
Event from an unknown agent: slack_agent, event id: b2adc65b-e6b5-4acb-b3b1-b93909b19c90
Event from an unknown agent: workflow_executor, event id: 5cde7bb9-7731-47b4-b318-1333a1929b7f
Event from an unknown agent: workflow_evaluator, event id: 251986e4-65ee-4dd5-9f

workflow_executor > {
  "plan": [
    {
      "action_index": 1,
      "agent": "sheets_agent",
      "parameters": {
        "row_values": [
          "323",
          "sinu",
          "hehe",
          "samajh gya"
        ]
      }
    }
  ],
  "simulation": {
    "workflow_name": "log_misc_issue",
    "total_steps": 1,
    "completed_steps": 1,
    "status": "completed",
    "log": [
      "Action 1: sheets_agent.add_row(row_values=['323', 'sinu', 'hehe', 'samajh gya'])"
    ]
  },
  "summary": "A row with details (323, sinu, hehe, samajh gya) was added to the spreadsheet."
}

 ### Continue session: debug_session_id

User > {"row_values": ["323", "sinu", "hehe", "samajh gya"]}


{
  "workflow": {
    "name": "log_misc_issue",
    "trigger": {
      "type": "intent_router",
      "source": "intent_router",
      "schedule": null,
      "conditions": [
        "intent_router said: automation"
      ]
    },
    "actions": [
      {
        "type": "add_row",
        "target": "spreadsheet",
        "description": "Add a row to the spreadsheet with issue details.",
        "inputs": [
          "323",
          "sinu",
          "hehe",
          "samajh gya"
        ]
      }
    ]
  },
  "evaluation": {
    "overall_score": 7,
    "verdict": "IMPROVE",
    "strengths": [
      "Clear Trigger: The workflow is triggered by the specific phrase 'intent_router said: automation', making its initiation explicit and understandable.",
      "Specific Action: The `add_row` action is well-defined, including all necessary inputs for the spreadsheet entry."
    ],
    "risks": [
      "Lack of Error Handling: The workflow does not include explicit error handling for the `ad

In [ ]:

# # Example: Automation (Slack)
# prompt = "Send a Slack notification to #new-channel now, saying I am kajal and grateful!"
# result = await handle_user_input(prompt, session_id="user_1")
# print(json.dumps(result, indent=2))


Event from an unknown agent: sheets_agent, event id: 66039ed2-7586-474a-b717-4e09f4ff1705
Event from an unknown agent: sheets_agent, event id: 8acaea2e-96b9-4369-a789-fd2985d5f534
Event from an unknown agent: sheets_agent, event id: 5be08b29-44b3-4ca2-8434-a7179d859eee
Event from an unknown agent: workflow_executor, event id: 995dc73b-f9a1-4109-8cd7-ccc05c0096a7
Event from an unknown agent: workflow_evaluator, event id: e876cd88-f3f9-48e8-a2a8-f271b7f90c7e
Event from an unknown agent: workflow_planner, event id: 9cafa20f-5205-4961-8194-d814afc79a88



 ### Continue session: debug_session_id

User > Send a Slack notification to #new-channel now, saying I am anchal and grateful!
intent_router > automation

 ### Continue session: debug_session_id

User > Send a Slack notification to #new-channel now, saying I am anchal and grateful!


Event from an unknown agent: intent_router, event id: 29e475a5-857b-408c-b4be-8c21fc4fdb25
Event from an unknown agent: sheets_agent, event id: 66039ed2-7586-474a-b717-4e09f4ff1705
Event from an unknown agent: sheets_agent, event id: 8acaea2e-96b9-4369-a789-fd2985d5f534
Event from an unknown agent: sheets_agent, event id: 5be08b29-44b3-4ca2-8434-a7179d859eee
Event from an unknown agent: workflow_executor, event id: 995dc73b-f9a1-4109-8cd7-ccc05c0096a7
Event from an unknown agent: workflow_evaluator, event id: e876cd88-f3f9-48e8-a2a8-f271b7f90c7e
Event from an unknown agent: workflow_planner, event id: 6fde0b4c-944c-432a-b978-9ae2e69f07dc
Event from an unknown agent: intent_router, event id: 29e475a5-857b-408c-b4be-8c21fc4fdb25
Event from an unknown agent: sheets_agent, event id: 66039ed2-7586-474a-b717-4e09f4ff1705
Event from an unknown agent: sheets_agent, event id: 8acaea2e-96b9-4369-a789-fd2985d5f534
Event from an unknown agent: sheets_agent, event id: 5be08b29-44b3-4ca2-8434-a7179d

workflow_planner > ```json
{
  "name": "gratitude_notification",
  "trigger": {
    "type": "agent_return",
    "source": "sheets_agent",
    "schedule": null,
    "conditions": [
      "sheets_agent.sheets_append_row returned success"
    ]
  },
  "actions": [
    {
      "type": "send_slack_message",
      "target": "#new-channel",
      "description": "Send a Slack notification to #new-channel with a gratitude message.",
      "inputs": [
        "I am anchal and grateful!"
      ]
    }
  ]
}
```

 ### Continue session: debug_session_id

User > {"name": "gratitude_notification", "trigger": {"type": "agent_return", "source": "sheets_agent", "schedule": null, "conditions": ["sheets_agent.sheets_append_row returned success"]}, "actions": [{"type": "send_slack_message", "target": "#new-channel", "description": "Send a Slack notification to #new-channel with a gratitude message.", "inputs": ["I am anchal and grateful!"]}]}


Event from an unknown agent: workflow_evaluator, event id: 251986e4-65ee-4dd5-9f07-d70bb4269da7
Event from an unknown agent: workflow_planner, event id: 6fde0b4c-944c-432a-b978-9ae2e69f07dc
Event from an unknown agent: intent_router, event id: 29e475a5-857b-408c-b4be-8c21fc4fdb25
Event from an unknown agent: sheets_agent, event id: 66039ed2-7586-474a-b717-4e09f4ff1705
Event from an unknown agent: sheets_agent, event id: 8acaea2e-96b9-4369-a789-fd2985d5f534
Event from an unknown agent: sheets_agent, event id: 5be08b29-44b3-4ca2-8434-a7179d859eee


workflow_evaluator > ```json
{
  "overall_score": 7,
  "verdict": "IMPROVE",
  "strengths": [
    "Clear Trigger: The workflow is triggered specifically by the successful completion of a previous agent action (`sheets_append_row`), ensuring it runs only when intended.",
    "Well-defined Action: The `send_slack_message` action clearly specifies the target channel and the message content."
  ],
  "risks": [
    "Hardcoded Identity: The message 'I am anchal and grateful!' hardcodes the sender as 'anchal', which is inflexible and may lead to misattribution if the workflow is used by different individuals.",
    "Fixed Target Channel: The Slack channel `#new-channel` is hardcoded, limiting reusability and requiring manual changes if the channel name is updated.",
    "Generic Message Content: The gratitude message is generic and lacks specific context about the event or action that prompted it, reducing its impact and informativeness.",
    "Privacy Concern: Hardcoding a user's identity ('

Event from an unknown agent: workflow_executor, event id: 5cde7bb9-7731-47b4-b318-1333a1929b7f
Event from an unknown agent: workflow_evaluator, event id: 251986e4-65ee-4dd5-9f07-d70bb4269da7
Event from an unknown agent: workflow_planner, event id: 6fde0b4c-944c-432a-b978-9ae2e69f07dc
Event from an unknown agent: intent_router, event id: 29e475a5-857b-408c-b4be-8c21fc4fdb25
Event from an unknown agent: sheets_agent, event id: 66039ed2-7586-474a-b717-4e09f4ff1705
Event from an unknown agent: sheets_agent, event id: 8acaea2e-96b9-4369-a789-fd2985d5f534
Event from an unknown agent: sheets_agent, event id: 5be08b29-44b3-4ca2-8434-a7179d859eee
Event from an unknown agent: workflow_executor, event id: 995dc73b-f9a1-4109-8cd7-ccc05c0096a7
Event from an unknown agent: workflow_evaluator, event id: e876cd88-f3f9-48e8-a2a8-f271b7f90c7e
Event from an unknown agent: workflow_planner, event id: 9cafa20f-5205-4961-8194-d814afc79a88
Event from an unknown agent: intent_router, event id: 1bc0f368-d2e8-4

{
  "workflow": {
    "name": "gratitude_notification",
    "trigger": {
      "type": "agent_return",
      "source": "sheets_agent",
      "schedule": null,
      "conditions": [
        "sheets_agent.sheets_append_row returned success"
      ]
    },
    "actions": [
      {
        "type": "send_slack_message",
        "target": "#new-channel",
        "description": "Send a Slack notification to #new-channel with a gratitude message.",
        "inputs": [
          "I am anchal and grateful!"
        ]
      }
    ]
  },
  "evaluation": {
    "overall_score": 7,
    "verdict": "IMPROVE",
    "strengths": [
      "Clear Trigger: The workflow is triggered specifically by the successful completion of a previous agent action (`sheets_append_row`), ensuring it runs only when intended.",
      "Well-defined Action: The `send_slack_message` action clearly specifies the target channel and the message content."
    ],
    "risks": [
      "Hardcoded Identity: The message 'I am anchal and g

In [ ]:

# # Example: Normal chat with memory
# prompt = "Hi, I am Kajal and I am a software engineer, I live in pune"
# await handle_user_input(prompt, session_id="user_1")
# # print(result)

# prompt = "Where do I live?"
# result = await handle_user_input(prompt, session_id="user_1")
# print(result)


Event from an unknown agent: sheets_agent, event id: 1d9e038e-5bfb-4d90-8e41-12d18b205c94
Event from an unknown agent: sheets_agent, event id: 1a739811-66ac-496f-8661-d9a554942169
Event from an unknown agent: sheets_agent, event id: 129fc6bc-8cf6-4a5d-9456-06bc7e82428b
Event from an unknown agent: workflow_executor, event id: a1f729d2-53a4-4de2-b082-8b271a220983
Event from an unknown agent: workflow_evaluator, event id: a549c3f5-905a-4f6a-8361-596370a290fa
Event from an unknown agent: workflow_planner, event id: 30b271ea-3173-4ea7-ad5b-27606c41c547



 ### Continue session: debug_session_id

User > Hi, I am Kajal and I am a software engineer, I live in pune
intent_router > chat

 ### Continue session: debug_session_id

User > Where do I live?
intent_router > chat
{'mode': 'chat', 'reply': 'You live in Pune.'}
